In [1]:
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Azure.Cosmos"
using Microsoft.Azure.Cosmos;
using System.Net.Http;

Installed Packages Microsoft.Azure.Cosmos, 3.53.1 Newtonsoft.Json, 13.0.4

In [2]:
var cstring = "AccountEndpoint=https://localhost:8081/;AccountKey=C2y6yDjf5/R+ob0N8A7Cgv30VRDJIWEHLM+4QDU5DE2nQ9nDuVTqobD4b8mGGyPMbIZnqyMsEcaGQy67XIw/Jw==";
var client = new CosmosClient(cstring);
var db = client.GetDatabase("Live");
var productContainer = db.GetContainer("Products");

In [3]:
public class Product    
{
    public string id { get; set; }
    public int ProductId { get; set; }
    public string Name { get; set; }
    public string Desciption { get; set; }
    public int categoryid { get; set; }
    public string CategoryName { get; set; }
    public decimal Price {get;set;}
    public bool InStock {get;set;}
    public List<string> ImageUrls { get; set; }        
}

id is the unique identifier for each doc within a logical partition.
id field is required
id must be string
id value must be less than 255 chars
Certain chars are not allowed in the id (/,\,?,#)
Try to make id and partition available for faster, cheaper reads.

In [4]:
var newproduct = new Product
{
    id = Guid.NewGuid().ToString(),
    ProductId = 1,
    Name = "Sample Product",
    Desciption = "This is a sample product.",
    categoryid = 1,
    CategoryName = "Category 1",
    Price = 19.99M,
    InStock = true,
    ImageUrls = new List<string> { "http://example.com/image1.jpg", "http://example.com/image2.jpg" }
};

Partition key is required to create a document.
Partition key can not be changed.

In [5]:
var newitem = await productContainer.CreateItemAsync(newproduct, new PartitionKey(newproduct.categoryid));
Console.WriteLine($"Created item in database with id: {newitem.Resource.id}");
Console.WriteLine($"Request charge: {newitem.RequestCharge}");

Created item in database with id: f61a1523-9d3a-4532-87d5-19e207215c13
Request charge: 8.95


Try to have partition key in your queries.
If id and partition key is available, prefer Point Read
You can retrieve results as a stream with GetItemQueryStreamIterator
Retrieve only what you need

In [8]:
var id = 1;
var productQuery = new QueryDefinition("SELECT * FROM c WHERE c.ProductId = @id")
    .WithParameter("@id", id);
var iterator = productContainer.GetItemQueryIterator<Product>(productQuery);
var results = new List<Product>();
while (iterator.HasMoreResults)
{
    var response = await iterator.ReadNextAsync();
    results.AddRange(response);
    Console.WriteLine($"Request charge: {response.RequestCharge}");
}
results[0].Display();

Request charge: 2.82


id,f61a1523-9d3a-4532-87d5-19e207215c13
ProductId,1
Name,Sample Product
Desciption,This is a sample product.
categoryid,1
CategoryName,Category 1
Price,19.99
InStock,True
ImageUrls,"[ http://example.com/image1.jpg, http://example.com/image2.jpg ]"


In [9]:

var item = await productContainer.ReadItemAsync<Product>(results[0].id, new PartitionKey(1));
item.Display();

Microsoft.Azure.Cosmos.ItemResponse`1[Submission#3+Product] Headers [ x-ms-activity-id, x-ms-last-state-change-utc, etag, x-ms-resource-quota, x-ms-resource-usage, x-ms-schemaversion, lsn, x-ms-request-charge, x-ms-alt-content-path, x-ms-content-path, x-ms-quorum-acked-lsn, x-ms-current-write-quorum, x-ms-current-replica-set-size, x-ms-documentdb-partitionkeyrangeid, x-ms-xp-role, x-ms-global-Committed-lsn, x-ms-number-of-read-regions, x-ms-item-lsn, x-ms-transport-request-id, x-ms-cosmos-llsn ... (more) ] ContinuationToken <null> RequestCharge 1 ActivityId 4c96d00c-87b1-4c61-997a-cda28380916f ETag "00000000-0000-0000-271c-aa826bfe01dc" ContentType <null> Session 0:-1#10 ContentLength <null> Location <null> (values) index value 0 x-ms-activity-id 1 x-ms-last-state-change-utc 2 etag 3 x-ms-resource-quota 4 x-ms-resource-usage 5 x-ms-schemaversion 6 lsn 7 x-ms-request-charge 8 x-ms-alt-content-path 9 x-ms-content-path 10 x-ms-quorum-acked-lsn 11 x-ms-current-write-quorum 12 x-ms-current-replica-set-size 13 x-ms-documentdb-partitionkeyrangeid 14 x-ms-xp-role 15 x-ms-global-Committed-lsn 16 x-ms-number-of-read-regions 17 x-ms-item-lsn 18 x-ms-transport-request-id 19 x-ms-cosmos-llsn ... (more) Resource Submission#3+Product id f61a1523-9d3a-4532-87d5-19e207215c13 ProductId 1 Name Sample Product Desciption This is a sample product. categoryid 1 CategoryName Category 1 Price 19.99 InStock True ImageUrls [ http://example.com/image1.jpg, http://example.com/image2.jpg ] StatusCode OK Diagnostics {"Summary":{"DirectCalls":{"(200, 0)":1}},"name":"ReadItemAsync","start datetime":"2025-09-16T15:25:08.859Z","duration in milliseconds":5.9325,"data":{"Client Configuration":{"Client Created Time Utc":"2025-09-16T13:53:05.2194139Z","MachineId":"hashedMachineName:854b5717-0c71-e584-b8c5-e106e5718dfa"... Value Microsoft.Azure.Cosmos.Tracing.Trace Name ReadItemAsync Id 48b2bb5a-2a68-439e-a2e6-85bd62835dbc StartTime 2025-09-16 15:25:08Z Duration 00:00:00.0059325 Level Info Component Transport Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary RegionsContacted index value 0 (South Central US, https://127.0.0.1:8081/) Item1 South Central US Item2 https://127.0.0.1:8081/ Parent <null> Children index value 0 Microsoft.Azure.Cosmos.Tracing.Trace Name Microsoft.Azure.Cosmos.Handlers.RequestInvokerHandler Id 5030fb26-c958-4bdf-8611-7f36c990519e StartTime 2025-09-16 15:25:08Z Duration 00:00:00.0043522 Level Info Component RequestHandler Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary RegionsContacted [ (South Central US, https://127.0.0.1:8081/) ] Parent Microsoft.Azure.Cosmos.Tracing.Trace Name ReadItemAsync Id 48b2bb5a-2a68-439e-a2e6-85bd62835dbc StartTime 9/16/2025 3:25:08 PM Duration 00:00:00.0059325 Level Info Component Transport Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary Parent <null> Children [ Microsoft.Azure.Cosmos.Tracing.Trace, Microsoft.Azure.Cosmos.Tracing.Trace ] Data [ [Client Configuration, Microsoft.Azure.Cosmos.Tracing.TraceData.ClientConfigurationTraceDatum] ] IsBeingWalked True Children index value 0 Microsoft.Azure.Cosmos.Tracing.Trace Data (empty) IsBeingWalked True 1 Microsoft.Azure.Cosmos.Tracing.Trace Name Response Serialization Id e470a0ad-5eaf-48cd-9191-0f3f7365bc23 StartTime 2025-09-16 15:25:08Z Duration 00:00:00.0000759 Level Verbose Component Transport Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary RegionsContacted [ (South Central US, https://127.0.0.1:8081/) ] Parent Microsoft.Azure.Cosmos.Tracing.Trace Name ReadItemAsync Id 48b2bb5a-2a68-439e-a2e6-85bd62835dbc StartTime 9/16/2025 3:25:08 PM Duration 00:00:00.0059325 Level Info Component Transport Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary Parent <null> Children [ Microsoft.Azure.Cosmos.Tracing.Trace, Microsoft.Azure.Cosmos.Tracing.Trace ] Data [ [Client Configuration, Microsoft.Azure.Cosmos.Tracing.TraceData.ClientConfigurationTraceDatum] ] IsBeingWalked True Children (empty) Data (empty) IsBeingWalked True Data key value Client Configuration Micr

Updates can be expensive, Consider batching if you need to update many documents

In [10]:
item.Resource.Desciption = "Updated";
var updated = await productContainer.ReplaceItemAsync(item.Resource, item.Resource.id, new PartitionKey(1)); 
updated.Display();


Microsoft.Azure.Cosmos.ItemResponse`1[Submission#3+Product] Headers [ x-ms-activity-id, x-ms-last-state-change-utc, etag, x-ms-resource-quota, x-ms-resource-usage, x-ms-schemaversion, lsn, x-ms-request-charge, x-ms-alt-content-path, x-ms-content-path, x-ms-quorum-acked-lsn, x-ms-current-write-quorum, x-ms-current-replica-set-size, x-ms-documentdb-partitionkeyrangeid, x-ms-xp-role, x-ms-global-Committed-lsn, x-ms-number-of-read-regions, x-ms-transport-request-id, x-ms-cosmos-llsn, x-ms-cosmos-quorum-acked-llsn ... (more) ] ContinuationToken <null> RequestCharge 10.67 ActivityId 1bfdb4b7-ee41-429f-b698-d9169d2a9312 ETag "00000000-0000-0000-271e-194fb33f01dc" ContentType <null> Session 0:-1#11 ContentLength <null> Location <null> (values) index value 0 x-ms-activity-id 1 x-ms-last-state-change-utc 2 etag 3 x-ms-resource-quota 4 x-ms-resource-usage 5 x-ms-schemaversion 6 lsn 7 x-ms-request-charge 8 x-ms-alt-content-path 9 x-ms-content-path 10 x-ms-quorum-acked-lsn 11 x-ms-current-write-quorum 12 x-ms-current-replica-set-size 13 x-ms-documentdb-partitionkeyrangeid 14 x-ms-xp-role 15 x-ms-global-Committed-lsn 16 x-ms-number-of-read-regions 17 x-ms-transport-request-id 18 x-ms-cosmos-llsn 19 x-ms-cosmos-quorum-acked-llsn ... (more) Resource Submission#3+Product id f61a1523-9d3a-4532-87d5-19e207215c13 ProductId 1 Name Sample Product Desciption Updated categoryid 1 CategoryName Category 1 Price 19.99 InStock True ImageUrls [ http://example.com/image1.jpg, http://example.com/image2.jpg ] StatusCode OK Diagnostics {"Summary":{"DirectCalls":{"(200, 0)":1}},"name":"ReplaceItemAsync","start datetime":"2025-09-16T15:25:14.139Z","duration in milliseconds":12.7522,"data":{"Client Configuration":{"Client Created Time Utc":"2025-09-16T13:53:05.2194139Z","MachineId":"hashedMachineName:854b5717-0c71-e584-b8c5-e106e5718... Value Microsoft.Azure.Cosmos.Tracing.Trace Name ReplaceItemAsync Id eee0bc43-c014-4d2c-a3fe-8fd0c5f8d0f1 StartTime 2025-09-16 15:25:14Z Duration 00:00:00.0127522 Level Info Component Transport Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary RegionsContacted index value 0 (South Central US, https://127.0.0.1:8081/) Item1 South Central US Item2 https://127.0.0.1:8081/ Parent <null> Children index value 0 Microsoft.Azure.Cosmos.Tracing.Trace Name ItemSerialize Id 37c83cd4-5051-43dc-9467-271d07cedbc7 StartTime 2025-09-16 15:25:14Z Duration 00:00:00.0000762 Level Verbose Component Transport Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary RegionsContacted [ (South Central US, https://127.0.0.1:8081/) ] Parent Microsoft.Azure.Cosmos.Tracing.Trace Name ReplaceItemAsync Id eee0bc43-c014-4d2c-a3fe-8fd0c5f8d0f1 StartTime 9/16/2025 3:25:14 PM Duration 00:00:00.0127522 Level Info Component Transport Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary Parent <null> Children [ Microsoft.Azure.Cosmos.Tracing.Trace, Microsoft.Azure.Cosmos.Tracing.Trace, Microsoft.Azure.Cosmos.Tracing.Trace ] Data [ [Client Configuration, Microsoft.Azure.Cosmos.Tracing.TraceData.ClientConfigurationTraceDatum] ] IsBeingWalked True Children (empty) Data (empty) IsBeingWalked True 1 Microsoft.Azure.Cosmos.Tracing.Trace Name Microsoft.Azure.Cosmos.Handlers.RequestInvokerHandler Id e522ab0b-370f-423c-ab73-80f72bae01d9 StartTime 2025-09-16 15:25:14Z Duration 00:00:00.0107912 Level Info Component RequestHandler Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary RegionsContacted [ (South Central US, https://127.0.0.1:8081/) ] Parent Microsoft.Azure.Cosmos.Tracing.Trace Name ReplaceItemAsync Id eee0bc43-c014-4d2c-a3fe-8fd0c5f8d0f1 StartTime 9/16/2025 3:25:14 PM Duration 00:00:00.0127522 Level Info Component Transport Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary Parent <null> Children [ Microsoft.Azure.Cosmos.Tracing.Trace, Microsoft.Azure.Cosmos.Tracing.Trace, Microsoft.Azure.Cosmos.Tracing.Trace ] Data [ [Client Configuration, Microsoft.Azure.Cosmos.Tracing.TraceData.ClientConfigurationTraceDatum] ] IsBeingWalked True Children index value 0 Microsoft.Azure

In [11]:
var updated2 = await productContainer.PatchItemAsync<Product>(item.Resource.id, new PartitionKey(1), new[] { PatchOperation.Replace("/Name", "Sample Product Updated") });
updated2.Display()

Microsoft.Azure.Cosmos.ItemResponse`1[Submission#3+Product] Headers [ x-ms-activity-id, x-ms-last-state-change-utc, etag, x-ms-resource-quota, x-ms-resource-usage, x-ms-schemaversion, lsn, x-ms-request-charge, x-ms-alt-content-path, x-ms-content-path, x-ms-quorum-acked-lsn, x-ms-current-write-quorum, x-ms-current-replica-set-size, x-ms-documentdb-partitionkeyrangeid, x-ms-xp-role, x-ms-global-Committed-lsn, x-ms-number-of-read-regions, x-ms-transport-request-id, x-ms-cosmos-llsn, x-ms-cosmos-quorum-acked-llsn ... (more) ] ContinuationToken <null> RequestCharge 10.79 ActivityId 056175bb-ed92-4349-977b-0d9005fb6382 ETag "00000000-0000-0000-271e-1f05192101dc" ContentType <null> Session 0:-1#12 ContentLength <null> Location <null> (values) index value 0 x-ms-activity-id 1 x-ms-last-state-change-utc 2 etag 3 x-ms-resource-quota 4 x-ms-resource-usage 5 x-ms-schemaversion 6 lsn 7 x-ms-request-charge 8 x-ms-alt-content-path 9 x-ms-content-path 10 x-ms-quorum-acked-lsn 11 x-ms-current-write-quorum 12 x-ms-current-replica-set-size 13 x-ms-documentdb-partitionkeyrangeid 14 x-ms-xp-role 15 x-ms-global-Committed-lsn 16 x-ms-number-of-read-regions 17 x-ms-transport-request-id 18 x-ms-cosmos-llsn 19 x-ms-cosmos-quorum-acked-llsn ... (more) Resource Submission#3+Product id f61a1523-9d3a-4532-87d5-19e207215c13 ProductId 1 Name Sample Product Updated Desciption Updated categoryid 1 CategoryName Category 1 Price 19.99 InStock True ImageUrls [ http://example.com/image1.jpg, http://example.com/image2.jpg ] StatusCode OK Diagnostics {"Summary":{"DirectCalls":{"(200, 0)":1}},"name":"PatchItemAsync","start datetime":"2025-09-16T15:25:23.700Z","duration in milliseconds":31.3576,"data":{"Client Configuration":{"Client Created Time Utc":"2025-09-16T13:53:05.2194139Z","MachineId":"hashedMachineName:854b5717-0c71-e584-b8c5-e106e5718df... Value Microsoft.Azure.Cosmos.Tracing.Trace Name PatchItemAsync Id 7471cae8-bb69-41f9-b655-6903d3b60db6 StartTime 2025-09-16 15:25:23Z Duration 00:00:00.0313576 Level Info Component Transport Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary RegionsContacted index value 0 (South Central US, https://127.0.0.1:8081/) Item1 South Central US Item2 https://127.0.0.1:8081/ Parent <null> Children index value 0 Microsoft.Azure.Cosmos.Tracing.Trace Name Patch Operations Serialize Id fd6ede5f-bab9-408e-877a-e3d37d00e589 StartTime 2025-09-16 15:25:23Z Duration 00:00:00.0034884 Level Verbose Component Transport Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary RegionsContacted [ (South Central US, https://127.0.0.1:8081/) ] Parent Microsoft.Azure.Cosmos.Tracing.Trace Name PatchItemAsync Id 7471cae8-bb69-41f9-b655-6903d3b60db6 StartTime 9/16/2025 3:25:23 PM Duration 00:00:00.0313576 Level Info Component Transport Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary Parent <null> Children [ Microsoft.Azure.Cosmos.Tracing.Trace, Microsoft.Azure.Cosmos.Tracing.Trace, Microsoft.Azure.Cosmos.Tracing.Trace ] Data [ [Client Configuration, Microsoft.Azure.Cosmos.Tracing.TraceData.ClientConfigurationTraceDatum] ] IsBeingWalked True Children (empty) Data (empty) IsBeingWalked True 1 Microsoft.Azure.Cosmos.Tracing.Trace Name Microsoft.Azure.Cosmos.Handlers.RequestInvokerHandler Id 436fd073-d69c-4489-ada1-b86467e9fe12 StartTime 2025-09-16 15:25:23Z Duration 00:00:00.0258458 Level Info Component RequestHandler Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary RegionsContacted [ (South Central US, https://127.0.0.1:8081/) ] Parent Microsoft.Azure.Cosmos.Tracing.Trace Name PatchItemAsync Id 7471cae8-bb69-41f9-b655-6903d3b60db6 StartTime 9/16/2025 3:25:23 PM Duration 00:00:00.0313576 Level Info Component Transport Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary Parent <null> Children [ Microsoft.Azure.Cosmos.Tracing.Trace, Microsoft.Azure.Cosmos.Tracing.Trace, Microsoft.Azure.Cosmos.Tracing.Trace ] Data [ [Client Configuration, Microsoft.Azure.Cosmos.Tracing.TraceData.ClientConfigurationTraceDatum] ] IsBeingWalked True Children index value 0 

In [12]:
var temp = await productContainer.DeleteItemAsync<Product>(item.Resource.id, new PartitionKey(1));
temp.Display();

Microsoft.Azure.Cosmos.ItemResponse`1[Submission#3+Product] Headers [ x-ms-activity-id, x-ms-last-state-change-utc, x-ms-resource-quota, x-ms-resource-usage, x-ms-schemaversion, lsn, x-ms-request-charge, x-ms-alt-content-path, x-ms-content-path, x-ms-quorum-acked-lsn, x-ms-current-write-quorum, x-ms-current-replica-set-size, x-ms-documentdb-partitionkeyrangeid, x-ms-xp-role, x-ms-global-Committed-lsn, x-ms-number-of-read-regions, x-ms-transport-request-id, x-ms-cosmos-llsn, x-ms-cosmos-quorum-acked-llsn, x-ms-session-token ... (more) ] ContinuationToken <null> RequestCharge 8.95 ActivityId d18d42c7-b85d-41ee-917b-35b9e019f468 ETag <null> ContentType <null> Session 0:-1#13 ContentLength <null> Location <null> (values) index value 0 x-ms-activity-id 1 x-ms-last-state-change-utc 2 x-ms-resource-quota 3 x-ms-resource-usage 4 x-ms-schemaversion 5 lsn 6 x-ms-request-charge 7 x-ms-alt-content-path 8 x-ms-content-path 9 x-ms-quorum-acked-lsn 10 x-ms-current-write-quorum 11 x-ms-current-replica-set-size 12 x-ms-documentdb-partitionkeyrangeid 13 x-ms-xp-role 14 x-ms-global-Committed-lsn 15 x-ms-number-of-read-regions 16 x-ms-transport-request-id 17 x-ms-cosmos-llsn 18 x-ms-cosmos-quorum-acked-llsn 19 x-ms-session-token ... (more) Resource <null> StatusCode NoContent Diagnostics {"Summary":{"DirectCalls":{"(204, 0)":1}},"name":"DeleteItemAsync","start datetime":"2025-09-16T15:25:29.771Z","duration in milliseconds":28.4617,"data":{"Client Configuration":{"Client Created Time Utc":"2025-09-16T13:53:05.2194139Z","MachineId":"hashedMachineName:854b5717-0c71-e584-b8c5-e106e5718d... Value Microsoft.Azure.Cosmos.Tracing.Trace Name DeleteItemAsync Id f427eb8d-abb4-4d7a-a452-20c9b29d9f7c StartTime 2025-09-16 15:25:29Z Duration 00:00:00.0284617 Level Info Component Transport Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary RegionsContacted index value 0 (South Central US, https://127.0.0.1:8081/) Item1 South Central US Item2 https://127.0.0.1:8081/ Parent <null> Children index value 0 Microsoft.Azure.Cosmos.Tracing.Trace Name Microsoft.Azure.Cosmos.Handlers.RequestInvokerHandler Id 072cf3d2-1629-4ed0-9645-1dd974f6b4b1 StartTime 2025-09-16 15:25:29Z Duration 00:00:00.0269066 Level Info Component RequestHandler Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary RegionsContacted [ (South Central US, https://127.0.0.1:8081/) ] Parent Microsoft.Azure.Cosmos.Tracing.Trace Name DeleteItemAsync Id f427eb8d-abb4-4d7a-a452-20c9b29d9f7c StartTime 9/16/2025 3:25:29 PM Duration 00:00:00.0284617 Level Info Component Transport Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary Parent <null> Children [ Microsoft.Azure.Cosmos.Tracing.Trace, Microsoft.Azure.Cosmos.Tracing.Trace ] Data [ [Client Configuration, Microsoft.Azure.Cosmos.Tracing.TraceData.ClientConfigurationTraceDatum] ] IsBeingWalked True Children index value 0 Microsoft.Azure.Cosmos.Tracing.Trace Data (empty) IsBeingWalked True 1 Microsoft.Azure.Cosmos.Tracing.Trace Name Response Serialization Id 135a3a3b-be2c-4457-a40e-db0f3a08085e StartTime 2025-09-16 15:25:29Z Duration 00:00:00.0000041 Level Verbose Component Transport Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary RegionsContacted [ (South Central US, https://127.0.0.1:8081/) ] Parent Microsoft.Azure.Cosmos.Tracing.Trace Name DeleteItemAsync Id f427eb8d-abb4-4d7a-a452-20c9b29d9f7c StartTime 9/16/2025 3:25:29 PM Duration 00:00:00.0284617 Level Info Component Transport Summary Microsoft.Azure.Cosmos.Tracing.TraceSummary Parent <null> Children [ Microsoft.Azure.Cosmos.Tracing.Trace, Microsoft.Azure.Cosmos.Tracing.Trace ] Data [ [Client Configuration, Microsoft.Azure.Cosmos.Tracing.TraceData.ClientConfigurationTraceDatum] ] IsBeingWalked True Children (empty) Data (empty) IsBeingWalked True Data key value Client Configuration Microsoft.Azure.Cosmos.Tracing.TraceData.ClientConfigurationTraceDatum ClientCreatedDateTimeUtc 2025-09-16 13:53:05Z GatewayConnectionConfig (cps:50, urto:6, p:False, httpf: False) MaxConnectionLimit 50 UserRequestTimeou